#### Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

   * если источник traffic_source равен yandex или google, то в source_type ставится organic
   * для источников paid и email из России - ставим ad
   * для источников paid и email не из России - ставим other
   * все остальные варианты берем из traffic_source без изменений

In [1]:
import pandas as pd

In [2]:
log = pd.read_csv('visit_log.csv', sep = ';')
log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [3]:
log.loc[(log.traffic_source.isin(['yandex','google'])), 'source_type'] = 'organic'
log.loc[((log.traffic_source == 'paid') | (log.traffic_source == 'email')) & (log.region =='Russia'), 'source_type'] = 'ad'
log.loc[((log.traffic_source == 'paid') | (log.traffic_source == 'email')) & (log.region !='Russia'), 'source_type'] = 'other'
log.loc[log.source_type.isnull(), 'source_type'] = log['traffic_source']
log['source_type'].value_counts()

direct     7013
organic    5407
ad         3790
other      2728
Name: source_type, dtype: int64

#### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

    1) Прочитайте содержимое файла с датафрейм
    2) Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [4]:
urls=pd.read_csv('URLs.txt')
urls[urls.url.str.contains('/[0-9]{8}-', regex=True)].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


#### Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [5]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
avg_life=ratings.groupby('userId').agg({'rating':'count','timestamp':['min','max']}).reset_index()
avg_life=avg_life[avg_life['rating']['count'] > 100]
avg_life.head()

userId rating   timestamp            
           count         min         max
3       4    204   949778714   949982274
7       8    116  1154389340  1154474527
14     15   1700   997937239  1469330735
16     17    363  1127468587  1127476640
18     19    423   855190091   855195373

In [7]:
def average(row):
    mean_ = row['timestamp']['max'] - row['timestamp']['min']
    return mean_

In [8]:
avg_life['avg']=avg_life.apply(average, axis=1)
avg_life[['userId', 'avg']].head()

,userId,avg
,,
3,4,203560
7,8,85187
14,15,471393496
16,17,8053
18,19,5282


#### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:

  *    таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
  *    аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [9]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [10]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [11]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [12]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [13]:
table=rzd.merge(auto, how='outer', on='client_id').merge(air, how='outer', on='client_id')
table.loc[table.rzd_revenue.isnull(), 'rzd_revenue']=0
table.loc[table.auto_revenue.isnull(), 'auto_revenue']=0
table.loc[table.air_revenue.isnull(), 'air_revenue']=0
table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [14]:
full_table=table.merge(client_base, how='outer', on='client_id')
full_table

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4
